#Prerequisite

In [0]:
df =spark.read.csv(path='/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/user_dataset/users_001.csv',
                   header=True,
                   inferSchema=True)
df.display()

#01. Transaction : Write data as Delta Format

In [0]:
df.write.format("delta").save("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta")

#02. Transaction -Overwrite Data Advantage

In [0]:
from pyspark.sql.functions import col
df.filter(col("country")== "India").write.save("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta",
                                               mode= "overwrite")

#read

In [0]:
df =spark.read.load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta")
df.display()

#Maintanes Version

In [0]:
spark.read.option("versionAsof",0).load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta").display()

##Meta data 
##approach1

In [0]:
spark.read.load(
    format="text",
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta/_delta_log/00000000000000000001.json",
).display()

#approach2

In [0]:
from delta.tables import DeltaTable
 
delta_table = DeltaTable.forPath(
    spark, "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta/"
)
delta_table.history().display()

# 03 Transaction - Upldate are allowed

In [0]:
df_delta = spark.read.load(
    "/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta/"
)
df_delta.createOrReplaceTempView("user_vw")

In [0]:
%sql

Update user_vw
SET country ="Bharat"
where country ="India"

In [0]:
df =spark.read.load("/Volumes/quickstart_catalog/quickstart_schema/sandbox/output_delta")
df.display()